In [ ]:
import nltk
nltk.download('brown')
nltk.download('stopwords')
nltk.download('punkt')
!pip3 install sent2vec
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Tweet sentiment analysis

Dataset
https://www.kaggle.com/kaushiksuresh147/the-social-dilemma-tweets

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
filepath = "/content/drive/MyDrive/Colab Notebooks/TheSocialDilemma.csv"

data = pd.read_csv(filepath)
data = data[['Sentiment', 'text']]
data.head(10)

,Sentiment,text
0,Neutral,@musicmadmarc @SocialDilemma_ @netflix @Facebo...
1,Neutral,@musicmadmarc @SocialDilemma_ @netflix @Facebo...
2,Positive,Go watch “The Social Dilemma” on Netflix!\n\nI...
3,Negative,I watched #TheSocialDilemma last night. I’m sc...
4,Positive,The problem of me being on my phone most the t...
5,Positive,#TheSocialDilemma 😳 wow!! We need regulations ...
6,Neutral,@harari_yuval what do you think about #TheSoci...
7,Negative,Erm #TheSocialDilemma makes me want to go off ...
8,Positive,"#TheSocialDilemma is not a documentary, it's h..."
9,Positive,Okay i’m watching #TheSocialDilemma now.


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

stop_words = set(stopwords.words('english'))
stop_words.update(['thank','damn','always','might','well','smfh','li','yall','u','r','nt','ok','i', 'must',
                 'please','knew','go','brb','m', 'even','much','yes','hi','wow','the', 'frm','ah','us','of', 
                 'on','also','us','okey','one', 'you', 'me', 'my', 'haa', 'erm','hey','okay', 'in', 'with', 
                 'and', 'we', 'don','day', 'amp','re'])

def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    tweet = re.sub('https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub(r'\@\w+|\#','', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    tweet = " ".join(filtered_words)

    return tweet

In [ ]:
data.text = data['text'].apply(preprocess_tweet_text)

data.head(10)

,Sentiment,text
0,Neutral,im reminded poignant quote french philosopher…
1,Neutral,marc get youre saying dont agree 🤪whicheve…
2,Positive,watch “ social dilemma ” netflixit ’ best 100 ...
3,Negative,watched thesocialdilemma last night ’ scared h...
4,Positive,problem phone time trying watch thesocialdilem...
5,Positive,thesocialdilemma 😳 need regulations social med...
6,Neutral,think thesocialdilemma netflix
7,Negative,thesocialdilemma makes want grid live cave tin...
8,Positive,thesocialdilemma documentary horror live
9,Positive,’ watching thesocialdilemma


In [ ]:
def get_feature_vector(train_fit):
    vector = TfidfVectorizer(sublinear_tf=True)
    vector.fit(train_fit)
    return vector

tf_vector = get_feature_vector(np.array(data.iloc[:, 1]).ravel())
X = tf_vector.transform(np.array(data.iloc[:, 1]).ravel())
y = np.array(data.iloc[:, 0]).ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

NB_model = MultinomialNB()
NB_model.fit(X_train, y_train)
y_predict_nb = NB_model.predict(X_test)
print(f1_score(y_test, y_predict_nb, average='weighted'))

LR_model = LogisticRegression(solver='lbfgs')
LR_model.fit(X_train, y_train)
y_predict_lr = LR_model.predict(X_test)
print(f1_score(y_test, y_predict_lr, average='weighted'))

0.633956122180819
0.8371599396285911


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
